# 指标组合策略

In [26]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt
import backtrader.indicator as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

现有指标：

+ 均线 
+ 布林带
+ MACD柱状图
+ RSI相对强弱
+ WilliamR指标

买入信号：指定某一单一因子发出买入信号时买入。

买入仓位：满仓买入。

卖出信号：指定某一单一因子发出卖出信号时卖出。

卖出仓位：一律平仓。

### 个股版本

In [86]:
class Composite_s(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('stake', 100),
        ('buy_signal', 'b'),
        ('sell_signal', 'b'),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('rsi_short', 6),
        ('rsi_long', 12),
        ('wave_period', 7),
        ('wr', 14),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))


    def __init__(self):
        # Initialization
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Alias
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Indicators
        ## Wave
        self.sma = btind.SMA(self.dataclose, period=self.p.wave_period)
        self.wave_buy = bt.And(self.sma > self.sma(-1), self.sma(-1) < self.sma(-2), self.sma(-2) < self.sma(-3))
        self.wave_sell = bt.And(self.sma < self.sma(-1), self.sma(-1) > self.sma(-2), self.sma(-2) > self.sma(-3))

        ## MACD
        self.macdhisto = btind.MACDHisto(self.dataclose)
        self.macd_buy = bt.And(self.macdhisto.histo > 0, self.macdhisto.histo(-1) < 0)
        self.macd_sell = bt.And(self.macdhisto.histo < 0, self.macdhisto.histo(-1) > 0)
        
        ## Bollinger Bands
        self.bolling = btind.BBands(self.dataclose, 
                                    period=self.p.bollinger_per, 
                                    devfactor=self.p.bollinger_dev)
        self.bolling_buy = bt.Or(bt.And(self.dataclose > self.bolling.top, self.dataclose(-1) < self.bolling.top(-1)), 
                                 bt.And(self.dataclose > self.bolling.bot, self.dataclose(-1) < self.bolling.bot(-1)))
        self.bolling_sell = bt.Or(bt.And(self.dataclose < self.bolling.top, self.dataclose(-1) > self.bolling.top(-1)), 
                                  bt.And(self.dataclose < self.bolling.bot, self.dataclose(-1) > self.bolling.bot(-1)))

        ## RSI
        self.rsi_s = btind.RSI(period=self.p.rsi_short, safediv=True)
        self.rsi_l = btind.RSI(period=self.p.rsi_long, safediv=True)
        self.rsi_buy = bt.And(self.rsi_s > self.rsi_l, self.rsi_s(-1) < self.rsi_l(-1))
        self.rsi_sell = bt.And(self.rsi_s < self.rsi_l, self.rsi_s(-1) > self.rsi_l(-1))

        ## WilliamR
        self.wr = btind.WilliamsR(period=self.p.wr)
        self.wr_buy = self.wr < -80
        self.wr_sell = self.wr > -20


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    

    def next(self):

        if self.order:
            return

        buy_indicator = (self.params.buy_signal == 'b' and self.bolling_buy) or \
                        (self.params.buy_signal == 'm' and self.macd_buy) or \
                        (self.params.buy_signal == 'w' and self.wave_buy and self.dataclose[0] > self.buyprice) or \
                        (self.params.buy_signal == 'r' and self.rsi_buy) or \
                        (self.params.buy_signal == 'wr' and self.wr_buy)
                        
        sell_indicator = (self.params.sell_signal == 'b' and self.bolling_sell) or \
                         (self.params.sell_signal == 'm' and self.macd_sell) or \
                         (self.params.sell_signal == 'w' and self.wave_sell) or \
                         (self.params.sell_signal == 'r' and self.rsi_sell) or \
                         (self.params.sell_signal == 'wr' and self.wr_sell)
            
    
        if buy_indicator and self.dataclose[0] > self.bolling.mid[0]: # Open position
            ava_pos = ((self.broker.get_cash() / self.p.units) // (self.dataclose[0]*self.p.stake)) * self.p.stake
            self.log('BUY CREATE, Price: %.2f, Lots: %i' % 
                     (self.dataclose[0], ava_pos))
            self.order = self.buy(size=ava_pos)
        elif sell_indicator and self.getposition(self.data).size > 0:
            self.log('Sell CREATE, Price: %.2f, Lots: %i' % 
                     (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()


    def stop(self):
        self.log('Ending Position:%i. Ending Value:%.2f.' %
                (self.getposition(self.data).size,
                self.cerebro.broker.getvalue()), doprint=False)

### 多股版本

In [ ]:
class Composite_m(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('stake', 100),
        ('buy_signal', 'b'),
        ('sell_signal', 'b'),

        # Indicator params
        ('bbands_per', 20),
        ('bbands_dev', 2),
        ('rsi_short', 6),
        ('rsi_long', 12),
        ('wave_period', 7),
        ('wr', 14),
    )

    
    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))

    
    def __init__(self):

        # Global initialization
        self.inds = dict()
        self.cash_unit = self.broker.get_cash() / self.p.units
        self.tracker = None

        for d in self.datas:
            # Local initialization
            self.inds[d] = dict()
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None
                
            # Indicators
            ## Wave
            self.inds[d]['sma'] = btind.SMA(d.close, period=self.p.wave_period)
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma'] > self.inds[d]['sma'](-1), 
                                              self.inds[d]['sma'](-1) < self.inds[d]['sma'](-2), 
                                              self.inds[d]['sma'](-2) < self.inds[d]['sma'](-3))
            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma'] < self.inds[d]['sma'](-1), 
                                               self.inds[d]['sma'](-1) > self.inds[d]['sma'](-2), 
                                               self.inds[d]['sma'](-2) > self.inds[d]['sma'](-3))

            ## MACD indicator
            self.inds[d]['macdhisto'] = btind.MACDHisto(d.close)
            self.inds[d]['macd_buy'] = bt.And(self.inds[d]['macdhisto'] > 0, self.inds[d]['macdhisto'](-1) < 0)
            self.inds[d]['macd_sell'] = bt.And(self.inds[d]['macdhisto'] < 0, self.inds[d]['macdhisto'](-1) > 0)

            ## Bollinger Bands indicator
            self.inds[d]['bbands'] = btind.BBands(d.close, 
                                                   period=self.p.bbands_per, 
                                                   devfactor=self.p.bbands_dev)
            self.inds[d]['bbands_buy'] = bt.Or(bt.And(d.close > self.inds[d]['bbands'].top, 
                                                       d.close(-1) < self.inds[d]['bbands'].top(-1)), 
                                                bt.And(d.close > self.inds[d]['bbands'].bot, 
                                                       d.close(-1) < self.inds[d]['bbands'].bot(-1)))
            self.inds[d]['bbands_sell'] = bt.Or(bt.And(d.close < self.inds[d]['bbands'].top, 
                                                        d.close(-1) > self.inds[d]['bbands'].top(-1)), 
                                                 bt.And(d.close < self.inds[d]['bbands'].bot, 
                                                        d.close(-1) > self.inds[d]['bbands'].bot(-1)))

            ## RSI indicator
            self.inds[d]['rsi_s'] = btind.RSI(d.close, period=self.p.rsi_short, safediv=True)
            self.inds[d]['rsi_l'] = btind.RSI(d.close, period=self.p.rsi_long, safediv=True)
            self.inds[d]['rsi_buy'] = bt.And(self.inds[d]['rsi_s'] > self.inds[d]['rsi_l'], 
                                             self.inds[d]['rsi_s'](-1) < self.inds[d]['rsi_l'](-1))
            self.inds[d]['rsi_sell'] = bt.And(self.inds[d]['rsi_s'] < self.inds[d]['rsi_l'], 
                                              self.inds[d]['rsi_s'](-1) > self.inds[d]['rsi_l'](-1))

            ## WilliamR indicator
            self.inds[d]['wr'] = btind.WilliamsR(d.close, period=self.p.wr)
            self.inds[d]['wr_buy'] = self.inds[d]['wr'] < -80
            self.inds[d]['wr_sell'] = self.inds[d]['wr'] > -20


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('买单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'], order.executed.price,
                          order.executed.size, self.broker.get_cash()))
                self.inds[self.tracker]['buyprice'] = order.executed.price

            else:  # Sell
                self.log('卖单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'], order.executed.price,
                         -order.executed.size, self.broker.get_cash()))
                self.inds[self.tracker]['sellprice'] = order.executed.price

        elif order.status in [order.Canceled]:
            self.log('订单取消：被撤销')

        elif order.status in [order.Margin]:
            self.log('订单取消：资金不足，代码：%s，持有资金:%i' %
                     (order.info['name'],
                     self.broker.get_cash()))

        elif order.status in [order.Rejected]:
            self.log('订单取消：被拒绝')
    
        self.inds[self.tracker]['order'] = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('交易收益 代码：%s, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.history[0]['event']['order'].info['name'], 
                  trade.pnl, trade.pnlcomm))
    

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
        # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return
            buy_indicator = (self.p.buy_signal == 'b' and self.inds[d]['bolling_buy']) or \
                            (self.p.buy_signal == 'm' and self.inds[d]['macd_buy']) or \
                            (self.p.buy_signal == 'w' and self.inds[d]['wave_buy'] and d.close[0] > self.inds[d]['buyprice']) or \
                            (self.p.buy_signal == 'r' and self.inds[d]['rsi_buy']) or \
                            (self.p.buy_signal == 'wr' and self.inds[d]['wr_buy'])


            sell_indicator = (self.p.sell_signal == 'b' and self.inds[d]['bolling_sell']) or \
                            (self.p.sell_signal == 'm' and self.inds[d]['macd_sell']) or \
                            (self.p.sell_signal == 'w' and self.inds[d]['wave_sell']) or \
                            (self.p.sell_signal == 'r' and self.inds[d]['rsi_sell']) or \
                            (self.p.sell_signal == 'wr' and self.inds[d]['wr_sell'])

            if sell_indicator and self.getposition(d).size > 0:
                sell_list.append((d, -1))

            elif buy_indicator and self.inds[d]['dataclose'][0] > self.inds[d]['bolling'].mid[0]:
                ava_pos = self.cash_unit // (self.p.stake * d.close[0]) * self.p.stake
                buy_list.append((d, ava_pos))
    
        for s_order in sell_list: # Close position
            self.tracker = s_order[0]
            self.log('卖单创建, 代码: %s, 价格: %.2f, 股数:%i' % 
                     (s_order[0]._name, s_order[0].close[0],
                      self.getposition(s_order[0]).size))
            self.inds[s_order[0]['order']] = self.close(data=s_order[0], name=s_order[0]._name)

        random.shuffle(buy_list)
        for b_order in buy_list:
            self.tracker = b_order[0]
            self.log('买单创建, 代码: %s, 价格: %.2f, 股数: %i' % 
                     (b_order[0]._name, b_order[0].close[0], 
                      b_order[1]))
            self.inds[s_order[0]['order']] = self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)

## 回测

### 单股回测

In [ ]:
stock_code = '601216.SH'
startdate = dt.datetime(2020,12,31) - dt.timedelta(days=385)
enddate = dt.datetime(2020,12,31)

if __name__ ==  '__main__':

    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(Composite_s, printlog=True, using_atr=True, atr_percent=0.02, buy_signal='b', sell_signal='m') 

    # Data
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
    cerebro.adddata(data)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)
    cerebro.broker.setcash(20000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    cerebro.plot(iplot=False)

### 单股循环回测调参

In [ ]:
stock_code = '601318.SH'
startdate = dt.datetime(2020,12,31) - dt.timedelta(days=365)
enddate = dt.datetime(2020,12,31)
buy_s = ['b','m','w', 'r', 'wr']
sell_s = ['b','m','w', 'r', 'wr']

if __name__ ==  '__main__':
    profit_dict = {}
    for s in product(*[buy_s, sell_s]):

        # Initialization
        cerebro = bt.Cerebro()
        strats = cerebro.addstrategy(Composite_s, printlog=False, using_atr=True, atr_percent=0.01, buy_signal=s[0], sell_signal=s[1]) 

        # Data
        df = utilsJ.get_stock(stock_code, startdate, enddate)
        data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
        cerebro.adddata(data)
        
        # Start condition
        cerebro.broker = bt.brokers.BackBroker(coc=True)   
        cerebro.broker.setcash(50000)
        #cerebro.broker.setcommission()
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Execution
        cerebro.run()

        # Final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
        profit_dict[s] = (final_value - start_value) / start_value * 100
    
    print(sorted(profit_dict.items(), key=lambda x: x[1], reverse=True))

### 沪深300循环回测
返回以沪深300为标的的股票对于指标策略的适配程度。

In [ ]:
index_code = '399300.SZ'
startdate = dt.datetime(2020,12,31) - dt.timedelta(days=365)
enddate = dt.datetime(2020,12,31)
index_list = utilsJ.get_index_components(index_code, startdate, enddate)
buy_s = ['b','m','w', 'r', 'wr']
sell_s = ['b','m','w', 'r', 'wr']
profit_stk = dict()

for stk in index_list:
    profit_ind = dict()
    for s in product(*[buy_s, sell_s]):

        # Initialization
        cerebro = bt.Cerebro()
        strats = cerebro.addstrategy(Composite_s, printlog=False, using_atr=True, atr_percent=0.02, buy_signal=s[0], sell_signal=s[1]) 

        # Data
        df = utilsJ.get_stock(stk, startdate, enddate)
        data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
        cerebro.adddata(data)

        # Start condition
        cerebro.broker = bt.brokers.BackBroker(coc=True)   
        cerebro.broker.setcash(1000000)
        #cerebro.broker.setcommission()
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Execution
        cerebro.run()

        # Print out the final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
        profit_ind[s] = (final_value - start_value) / start_value * 100
    print(sorted(profit_ind.items(), key=lambda x: x[1], reverse=True)[:3])
    profit_stk[stk] = sorted(profit_ind.items(), key=lambda x: x[1], reverse=True)

### 多股回测

依旧采用沪深300成分股作为标的

In [ ]:
random.seed(10086)
startdate = dt.datetime(2020, 1, 1) - dt.timedelta(days=20)
enddate = dt.datetime(2020, 12, 31)
index_code = '399300.SZ'
index_list = utilsJ.get_index_components(index_code, startdate, enddate)

if __name__ == '__main__':
    
    # Initialization
    cerebro = bt.Cerebro(tradehistory=True)
    strats = cerebro.addstrategy(Composite_m, printlog=True, buy_signal='wr', sell_signal='m') 

    # Data
    for stk in index_list:
        df = utilsJ.get_stock(stk, startdate, enddate)
        data = bt.feeds.PandasData(dataname=df,fromdate=startdate,todate=enddate)
        cerebro.adddata(data, name=stk)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)
    cerebro.broker.setcash(1000000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()

    # Execution
    cerebro.run(maxcpus=8)

    # Final result
    final_value = cerebro.broker.getvalue()
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    #profit.append((final_value - start_value) / start_value * 100)

### 多股循环回测调参

In [ ]:
startdate = dt.datetime(2020, 1, 1) - dt.timedelta(days = 20)
enddate = dt.datetime(2020, 12, 31)
index_code = '399300.SZ'
index_list = utilsJ.get_index_components(index_code, startdate, enddate)
buy_s = ['b','m','w', 'r', 'wr']
sell_s = ['b','m','w', 'r', 'wr']


if __name__ == '__main__':
    profit_dict = {}
    for s in product(*[buy_s, sell_s]):
        random.seed(10086)
        for i in range(3):

            # Initialization
            cerebro = bt.Cerebro(tradehistory=True)
            strats = cerebro.addstrategy(Composite_m, printlog=False, buy_signal=s[0], sell_signal=s[1])

            # Data
            for stk in index_list:
                df = utilsJ.get_stock(stk, startdate, enddate)
                data = bt.feeds.PandasData(dataname=df, fromdate=startdate, todate=enddate)
                cerebro.adddata(data, name=stk)
            
            # Start conditions
            cerebro.broker = bt.brokers.BackBroker(coc=True)   
            cerebro.broker.setcash(1000000)
            #cerebro.broker.setcommission()
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Execution
            cerebro.run(maxcpus=8)

            # Final result
            final_value = cerebro.broker.getvalue()
            #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
            print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value - start_value) / start_value * 100))
            if (s[0], s[1]) not in profit_dict.keys():
                profit_dict[(s[0], s[1])] = []
            profit_dict[(s[0], s[1])].append((final_value - start_value) / start_value * 100)